Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, welch
from scipy import stats
import mne
from sklearn.utils import resample
from mne.filter import filter_data
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,cross_validate, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score, precision_score,recall_score, f1_score, confusion_matrix, make_scorer

Summary file parsing to extract seizure start and end times

In [2]:
def extract_seizure_events_from_txt(folder_path):
    """
    Parses all .txt summary files in a folder to extract seizure start/end times
    for each corresponding .edf file.

    """
    seizure_info = {}

    txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    if not txt_files:
        raise FileNotFoundError("No .txt summary files found in the folder.")

    for txt_file in txt_files:
        summary_file = os.path.join(folder_path, txt_file)
        current_file = None
        current_start = None

        with open(summary_file, "r") as f:
            for line in f:
                line = line.strip()

                if line.startswith("File Name:"):
                    current_file = line.split(":", 1)[1].strip()
                    if current_file not in seizure_info:
                        seizure_info[current_file] = []

                elif line.startswith("Seizure Start Time:"):
                    current_start = int(line.split(":", 1)[1].strip().split()[0])

                elif line.startswith("Seizure End Time:") and current_start is not None:
                    end_time = int(line.split(":", 1)[1].strip().split()[0])
                    seizure_info[current_file].append((current_start, end_time))
                    current_start = None 

    return seizure_info


Data Loading

In [3]:
def load_edf_with_seizures(edf_path, seizure_times, sampling_rate=256):
    """
    Load EDF file and return data with seizure annotations.
    Keeps only first occurrence of duplicate base channel names.
    """
    raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)

    base_names = [name.split('-')[0] + '-' + name.split('-')[1] if name.count('-') >= 2 else name for name in raw.ch_names]

    seen = set()
    keep = []
    for i, name in enumerate(base_names):
        if name not in seen:
            seen.add(name)
            keep.append(raw.ch_names[i])

    raw.pick_channels(keep)

    seizure_samples = [
        (int(start * sampling_rate), int(end * sampling_rate))
        for start, end in seizure_times
    ]

    return {
        'raw': raw,
        'seizure_samples': seizure_samples
    }


In [4]:
def batch_load_edf_with_seizures(data_folder):
    """
    Loads all .edf files in a folder with their seizure annotations.

    """
    seizure_info = extract_seizure_events_from_txt(data_folder)
    loaded_data = {}

    for fname in os.listdir(data_folder):
        if fname.endswith('.edf') and fname in seizure_info:
            edf_path = os.path.join(data_folder, fname)
            seizure_times = seizure_info[fname]

            try:
                result = load_edf_with_seizures(edf_path, seizure_times)
                loaded_data[fname] = result
                print(f"Loaded {fname} with {len(result['seizure_samples'])} seizure intervals")
            except Exception as e:
                print(f"Failed to load {fname}: {e}")

    return loaded_data

Data Processing

In [5]:
def process_single_file(raw, edf_filename, output_folder, selected_channels, seizure_windows):
    """
    Preprocesses EEG and saves .npz with metadata.
    """
    os.makedirs(output_folder, exist_ok=True)

    raw.pick_channels(selected_channels)
    data = raw.get_data()
    sfreq = raw.info['sfreq']


    data = filter_data(data, sfreq=sfreq, l_freq=0.5, h_freq=25.0, verbose=False)

    base_name = os.path.splitext(edf_filename)[0]
    save_path = os.path.join(output_folder, f"{base_name}_preprocessed.npz")
    np.savez(save_path,
             data=data,
             seizure_windows=np.array(seizure_windows, dtype=np.int32),
             sampling_rate=sfreq,
             channels=np.array(selected_channels),
             file_name=edf_filename)

    return data

In [6]:
def preprocess_and_save(edf_folder):
    """
    Batch preprocesses and saves all EDF files in folder.
    Skips files missing required channels.
    """
    output_folder = os.path.join(edf_folder, 'preprocessed')
    os.makedirs(output_folder, exist_ok=True)

    selected_channels = [
        'FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1',
        'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1',
        'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2',
        'FP2-F8', 'F8-T8',  'P8-O2',
        'FZ-CZ', 'CZ-PZ'
    ]

    seizure_dict = extract_seizure_events_from_txt(edf_folder)

    for fname in os.listdir(edf_folder):
        if fname.endswith('.edf'):
            edf_path = os.path.join(edf_folder, fname)
            seizure_times = seizure_dict.get(fname, [])

            try:
                # Load with filtering of duplicate channels
                data_obj = load_edf_with_seizures(edf_path, seizure_times)
                raw = data_obj['raw']

                # Check if all selected channels are present (after cleaning)
                available_channels = set(raw.ch_names)
                if not all(chan in available_channels for chan in selected_channels):
                    print(f"Skipping {fname}: missing expected channels.")
                    continue

                # Now process and save
                process_single_file(
                    raw=raw,
                    edf_filename=fname,
                    output_folder=output_folder,
                    selected_channels=selected_channels,
                    seizure_windows=data_obj['seizure_samples']
                )

                base_name = os.path.splitext(fname)[0]
                print(f"Saved to: {os.path.join(output_folder, base_name + '_preprocessed.npz')}\n")

            except Exception as e:
                print(f"Failed to process {fname}: {e}")


In [7]:
def segment_and_label(eeg_data, seizure_windows, window_duration=2.0, sampling_rate=256, overlap=0.5):
    """
    Segments EEG into overlapping windows and labels them
    """
    window_size = int(window_duration*sampling_rate)
    step_size = int(window_size*(1-overlap))
    channels, total_samples = eeg_data.shape
    X,y = [], []
    for start in range(0, total_samples-window_size + 1, step_size):
        end = start + window_size
        window = eeg_data[:, start:end]

        label = 0
        for sz_start, sz_end in seizure_windows:
            if end > sz_start and start < sz_end:
                label = 1
                break
        X.append(window) 
        y.append(label)

    X = np.stack(X)
    y = np.stack(y)

    return X,y

In [8]:
def batch_segment_preprocessed(folder_path):
    """
    Segments all preprocessed .npz files in 'preprocessed/' subfolder
    into 2s windows and labels them.
    """
    preprocessed_folder = os.path.join(folder_path, 'preprocessed')
    output_folder = os.path.join(folder_path, 'segmented')
    os.makedirs(output_folder, exist_ok=True)

    for fname in os.listdir(preprocessed_folder):
        if fname.endswith("_preprocessed.npz"):
            full_path = os.path.join(preprocessed_folder, fname)

            try:
                npz = np.load(full_path, allow_pickle=True)
                eeg = npz['data']
                sz_windows = npz["seizure_windows"]
                sfreq = int(npz['sampling_rate'])
                file_name = str(npz['file_name'])
                channels = list(npz['channels'])
                X, y = segment_and_label(eeg_data=eeg,
                         seizure_windows=sz_windows,
                         window_duration=2.0,
                         sampling_rate=sfreq,
                         overlap=0.5)
                base_name = fname.replace('_preprocessed.npz', '')
                save_path = os.path.join(output_folder, f"{base_name}_segmented.npz")
                np.savez(save_path,
                         X=X,
                         y=y,
                         sampling_rate=sfreq,
                         channels=channels,
                         file_name=file_name)

                print(f"Segmented and saved: {save_path}")

            except Exception as e:
                print(f"Failed on {fname}: {e}")


Concatenate all NPZ files

In [9]:

def load_segmented_dataset(folder_path):
    """
    Loads and concatenates all segmented .npz files in a folder.
    """
    X_list, y_list = [], []

    for fname in os.listdir(folder_path):
        if fname.endswith('_segmented.npz'):
            npz_path = os.path.join(folder_path, fname)
            try:
                npz = np.load(npz_path)
                X = npz['X']
                y = npz['y']
                X_list.append(X)
                y_list.append(y)
            except Exception as e:
                print(f"Could not load {fname}: {e}")

    X_total = np.concatenate(X_list, axis=0)
    y_total = np.concatenate(y_list, axis=0)

    return X_total, y_total

############ Calling The Functions ############

In [10]:
extract_seizure_events_from_txt("C:\\Users\\itama\\Desktop\\data analysis for neuroscience course\\Seizure_Onset_Prediction\\data")

{'chb01_01.edf': [],
 'chb01_02.edf': [],
 'chb01_03.edf': [(2996, 3036)],
 'chb01_04.edf': [(1467, 1494)],
 'chb01_05.edf': [],
 'chb01_06.edf': [],
 'chb01_07.edf': [],
 'chb01_08.edf': [],
 'chb01_09.edf': [],
 'chb01_10.edf': [],
 'chb01_11.edf': [],
 'chb01_12.edf': [],
 'chb01_13.edf': [],
 'chb01_14.edf': [],
 'chb01_15.edf': [(1732, 1772)],
 'chb01_16.edf': [(1015, 1066)],
 'chb01_17.edf': [],
 'chb01_18.edf': [(1720, 1810)],
 'chb01_19.edf': [],
 'chb01_20.edf': [],
 'chb01_21.edf': [(327, 420)],
 'chb01_22.edf': [],
 'chb01_23.edf': [],
 'chb01_24.edf': [],
 'chb01_25.edf': [],
 'chb01_26.edf': [(1862, 1963)],
 'chb01_27.edf': [],
 'chb01_29.edf': [],
 'chb01_30.edf': [],
 'chb01_31.edf': [],
 'chb01_32.edf': [],
 'chb01_33.edf': [],
 'chb01_34.edf': [],
 'chb01_36.edf': [],
 'chb01_37.edf': [],
 'chb01_38.edf': [],
 'chb01_39.edf': [],
 'chb01_40.edf': [],
 'chb01_41.edf': [],
 'chb01_42.edf': [],
 'chb01_43.edf': [],
 'chb01_46.edf': [],
 'chb02_01.edf': [],
 'chb02_02.edf'

In [11]:
batch_load_edf_with_seizures("C:\\Users\\itama\\Desktop\\data analysis for neuroscience course\\Seizure_Onset_Prediction\\data")

C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Loaded chb01_03.edf with 1 seizure intervals


C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Loaded chb01_04.edf with 1 seizure intervals


C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


...

C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Loaded chb02_19.edf with 1 seizure intervals


C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Loaded chb02_35.edf with 0 seizure intervals


{'chb01_03.edf': {'raw': <RawEDF | chb01_03.edf, 22 x 921600 (3600.0 s), ~154.7 MiB, data loaded>,
  'seizure_samples': [(766976, 777216)]},
 'chb01_04.edf': {'raw': <RawEDF | chb01_04.edf, 22 x 921600 (3600.0 s), ~154.7 MiB, data loaded>,
  'seizure_samples': [(375552, 382464)]},
 'chb01_15.edf': {'raw': <RawEDF | chb01_15.edf, 22 x 921600 (3600.0 s), ~154.7 MiB, data loaded>,
  'seizure_samples': [(443392, 453632)]},
 'chb01_16.edf': {'raw': <RawEDF | chb01_16.edf, 22 x 921600 (3600.0 s), ~154.7 MiB, data loaded>,
  'seizure_samples': [(259840, 272896)]},
 'chb01_18.edf': {'raw': <RawEDF | chb01_18.edf, 22 x 921600 (3600.0 s), ~154.7 MiB, data loaded>,
  'seizure_samples': [(440320, 463360)]},
 'chb01_21.edf': {'raw': <RawEDF | chb01_21.edf, 22 x 921600 (3600.0 s), ~154.7 MiB, data loaded>,
  'seizure_samples': [(83712, 107520)]},
 'chb01_26.edf': {'raw': <RawEDF | chb01_26.edf, 22 x 595200 (2325.0 s), ~99.9 MiB, data loaded>,
  'seizure_samples': [(476672, 502528)]},
 'chb02_05.edf'

In [12]:
preprocess_and_save("C:\\Users\\itama\\Desktop\\data analysis for neuroscience course\\Seizure_Onset_Prediction\\data")

C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Saved to: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\preprocessed\chb01_03_preprocessed.npz



C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Saved to: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\preprocessed\chb01_04_preprocessed.npz



C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Saved to: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\preprocessed\chb02_16+_preprocessed.npz



C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Saved to: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\preprocessed\chb02_19_preprocessed.npz



C:\Users\itama\AppData\Local\Temp\ipykernel_12904\2455710421.py:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_path, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Saved to: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\preprocessed\chb02_35_preprocessed.npz



In [13]:
batch_segment_preprocessed("C:\\Users\\itama\\Desktop\\data analysis for neuroscience course\\Seizure_Onset_Prediction\\data")

Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_03_segmented.npz
Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_04_segmented.npz
Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_15_segmented.npz
Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_16_segmented.npz
Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_18_segmented.npz
Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_21_segmented.npz
Segmented and saved: C:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\data\segmented\chb01_26

In [14]:
X_total, y_total = load_segmented_dataset("C:\\Users\\itama\\Desktop\\data analysis for neuroscience course\\Seizure_Onset_Prediction\\data\\segmented")

Training and Evaluation

In [15]:


def downsample_majority(X, y, majority_label=0, minority_label=1, majority_n=1500):
    """
    Downsamples the majority class and returns the balanced dataset.
    """
 
    X_major = X[y == majority_label]
    y_major = y[y == majority_label]
    X_minor = X[y == minority_label]
    y_minor = y[y == minority_label]


    X_major_down, y_major_down = resample(
        X_major, y_major,
        replace=False,
        n_samples=majority_n,
        random_state=42
    )

    X_balanced = np.concatenate([X_major_down, X_minor], axis=0)
    y_balanced = np.concatenate([y_major_down, y_minor], axis=0)

    return X_balanced, y_balanced


In [16]:
X_bal, y_bal = downsample_majority(X_total, y_total)


In [26]:
X_total

array([[[ 2.87991202e-20,  6.42969641e-06,  1.22599718e-05, ...,
          1.56281909e-05,  1.14660171e-05,  5.43074258e-06],
        [-1.01643954e-20, -1.56614291e-05, -2.92070716e-05, ...,
          9.52223067e-06,  2.84919028e-06, -3.36021556e-06],
        [-1.52465931e-20,  1.19023816e-06,  2.33426152e-06, ...,
         -5.36879916e-06, -6.52776753e-06, -6.50054765e-06],
        ...,
        [-4.06575815e-20, -8.09525175e-05, -1.47461516e-04, ...,
         -2.48980473e-05, -2.11648211e-05, -1.67908048e-05],
        [-2.54109884e-20,  6.95159930e-06,  1.31081691e-05, ...,
          3.11578356e-06,  1.20153036e-06,  8.49901912e-07],
        [-1.69406589e-20, -3.64806395e-05, -6.70565824e-05, ...,
         -2.73760954e-05, -2.99965436e-05, -3.18070190e-05]],

       [[ 4.14498987e-05,  4.47206844e-05,  4.65753781e-05, ...,
         -1.19632218e-05, -1.18090963e-07,  1.18764902e-05],
        [ 9.43266886e-06,  8.44705847e-06,  7.52008485e-06, ...,
          4.69829433e-05,  4.83797413e

In [18]:

def specificity_score(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    tn = cm[0, 0]
    fp = cm[0, 1]
    return tn / (tn + fp + 1e-10)

def train_ml(X_total, y_total):
    """
    Trains classical ML models (Logistic Regression, Random Forest, SVM) on flattened EEG data.
    """
    X_total_flat = X_total.reshape(X_total.shape[0], -1)
    X_train, X_temp, y_train, y_temp = train_test_split(X_total_flat, y_total, test_size=0.3, stratify=y_total, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

    models = {
        # 'LogisticRegression': LogisticRegression(class_weight="balanced", max_iter=1000),
        # 'RandomForest': RandomForestClassifier(n_estimators=100),
        'SVM': SVC()
    }

    scoring = {
        'accuracy': 'accuracy',
        'precision': 'precision',
        'recall': 'recall',
        'f1': 'f1'
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    trained_models = {}

    for name, model in models.items():
        print(f"\n==== {name} ====")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('clf', model)
        ])

        cv_results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring=scoring)
        for metric in scoring:
            scores = cv_results[f'test_{metric}']
            print(f"{metric.capitalize()} (CV): {scores.mean():.4f} ± {scores.std():.4f}")

        pipeline.fit(X_train, y_train)
        trained_models[name] = pipeline  # store fitted model
        y_val_pred = pipeline.predict(X_val)

        acc = accuracy_score(y_val, y_val_pred)
        prec = precision_score(y_val, y_val_pred)
        rec = recall_score(y_val, y_val_pred)
        spec = specificity_score(y_val, y_val_pred)
        f1 = f1_score(y_val, y_val_pred)

        print(f"\nValidation Set Performance:")
        print(f"Accuracy:    {acc:.4f}")
        print(f"Precision:   {prec:.4f}")
        print(f"Recall:      {rec:.4f}")
        print(f"Specificity: {spec:.4f}")
        print(f"F1 Score:    {f1:.4f}")

        y_test_pred = pipeline.predict(X_test)
        acc = accuracy_score(y_test, y_test_pred)
        prec = precision_score(y_test, y_test_pred)
        rec = recall_score(y_test, y_test_pred)
        spec = specificity_score(y_test, y_test_pred)
        f1 = f1_score(y_test, y_test_pred)

        print(f"\nTest Set Performance:")
        print(f"Accuracy:    {acc:.4f}")
        print(f"Precision:   {prec:.4f}")
        print(f"Recall:      {rec:.4f}")
        print(f"Specificity: {spec:.4f}")
        print(f"F1 Score:    {f1:.4f}")
    return trained_models, (X_train, X_val, X_test, y_train, y_val, y_test)



In [19]:
trained_models, splits = train_ml(X_total, y_total)
X_train, X_val, X_test, y_train, y_val, y_test = splits


==== SVM ====


c:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no pre

Accuracy (CV): 0.9871 ± 0.0001
Precision (CV): 0.0000 ± 0.0000
Recall (CV): 0.0000 ± 0.0000
F1 (CV): 0.0000 ± 0.0000


c:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])



Validation Set Performance:
Accuracy:    0.9871
Precision:   0.0000
Recall:      0.0000
Specificity: 1.0000
F1 Score:    0.0000

Test Set Performance:
Accuracy:    0.9871
Precision:   0.0000
Recall:      0.0000
Specificity: 1.0000
F1 Score:    0.0000


c:\Users\itama\Desktop\data analysis for neuroscience course\Seizure_Onset_Prediction\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [21]:
#saving the balanced data
np.savez_compressed("balanced_dataset1.npz", X=X_bal, y=y_bal)


#Optimizing the model

In [22]:
#hyperparamter tuning with pca

models = {
    'LogisticRegression': LogisticRegression(class_weight='balanced', max_iter=1000),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(class_weight='balanced')}

param_grid = {
    'LogisticRegression': {'pca__n_components': [10, 20, 30, 50],'clf__C': [0.01, 0.1, 1, 10], 'clf__penalty': ['l2'], 'clf__solver': ['lbfgs']},
    'RandomForest': {'pca__n_components': [10, 20, 30, 50],'clf__n_estimators': [100,200], 'clf__max_depth': [None, 10, 20]},
    'SVM': {'pca__n_components': [10, 20, 30, 50],'clf__C': [0.1,1,10], 'clf__kernel': ['rbf', 'linear'], 'clf__gamma': ['scale', 'auto']}}


def train_ml_optimized(X_total, y_total):
    """
    Here is the optimized training function for the models
    """
    X_total_flat = X_total.reshape(X_total.shape[0], -1)
    X_train, X_temp, y_train, y_temp = train_test_split(X_total_flat, y_total, test_size=0.3, stratify=y_total, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for name in models:
        print(f"\n==== {name} ====")
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA()),
            ('clf', models[name])
        ])
        grid = GridSearchCV(
            pipeline,
            param_grid[name],
            cv=cv,
            scoring='f1',
            n_jobs=-1
        )
        grid.fit(X_train, y_train)
        best_model = grid.best_estimator_
        print(f"Best Params: {grid.best_params_}")

        y_val_pred = best_model.predict(X_val)
        print("\nValidation Set Performance:")
        print(f"Accuracy:    {accuracy_score(y_val, y_val_pred):.4f}")
        print(f"Precision:   {precision_score(y_val, y_val_pred):.4f}")
        print(f"Recall:      {recall_score(y_val, y_val_pred):.4f}")
        print(f"Specificity: {specificity_score(y_val, y_val_pred):.4f}")
        print(f"F1 Score:    {f1_score(y_val, y_val_pred):.4f}")

        y_test_pred = best_model.predict(X_test)
        print("\nTest Set Performance:")
        print(f"Accuracy:    {accuracy_score(y_test, y_test_pred):.4f}")
        print(f"Precision:   {precision_score(y_test, y_test_pred):.4f}")
        print(f"Recall:      {recall_score(y_test, y_test_pred):.4f}")
        print(f"Specificity: {specificity_score(y_test, y_test_pred):.4f}")
        print(f"F1 Score:    {f1_score(y_test, y_test_pred):.4f}")

  
    


In [23]:
train_ml_optimized(X_bal, y_bal)


==== LogisticRegression ====
Best Params: {'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs', 'pca__n_components': 20}

Validation Set Performance:
Accuracy:    0.6242
Precision:   0.3300
Recall:      0.4074
Specificity: 0.7022
F1 Score:    0.3646

Test Set Performance:
Accuracy:    0.5993
Precision:   0.2857
Recall:      0.3457
Specificity: 0.6903
F1 Score:    0.3128

==== RandomForest ====
Best Params: {'clf__max_depth': None, 'clf__n_estimators': 200, 'pca__n_components': 50}

Validation Set Performance:
Accuracy:    0.9020
Precision:   0.8493
Recall:      0.7654
Specificity: 0.9511
F1 Score:    0.8052

Test Set Performance:
Accuracy:    0.9283
Precision:   0.9155
Recall:      0.8025
Specificity: 0.9735
F1 Score:    0.8553

==== SVM ====


KeyboardInterrupt: 

Visualizations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc

def _get_scores_for_binary_roc(model, X):
    """
    Returns a continuous score for the positive class:
    - decision_function if available
    - otherwise the probability for class 1 (or the 'positive' label if provided by caller)
    """
    if hasattr(model, "decision_function"):
        s = model.decision_function(X)
        # Some estimators return shape (n_samples, n_classes). Reduce to positive class if binary.
        if s.ndim == 2 and s.shape[1] == 2:
            s = s[:, 1]
        return s
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba(X)
        # assume binary, take probability of the 2nd column (positive class)
        if proba.shape[1] == 2:
            return proba[:, 1]
    raise ValueError("Model must support decision_function or predict_proba for ROC.")

# Confusion Matrix ----
def plot_confusion_matrix_svm(y_true, y_pred, labels=None, normalize='true', title="SVM Confusion Matrix"):
    """
    normalize: {'true','pred','all', None} mirrors sklearn
    labels: optional sequence of label names in the order you want to display
    """
    cm = confusion_matrix(y_true, y_pred, labels=labels, normalize=normalize)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels if labels is not None else None)
    fig, ax = plt.subplots(figsize=(5.5, 5))
    disp.plot(ax=ax, cmap="Blues", colorbar=True, values_format=".2f" if normalize else "d")
    ax.set_title(title)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.tight_layout()
    plt.show()

# ROC Curve + AUC (binary) ----
def plot_roc_auc_svm(y_true, y_score, pos_label=1, title="SVM ROC Curve"):
    """
    y_score: continuous score for the positive class (from decision_function or predict_proba[:,1])
    pos_label: which label counts as the positive class in y_true
    """
    fpr, tpr, _ = roc_curve(y_true, y_score, pos_label=pos_label)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(5.5, 5))
    plt.plot(fpr, tpr, linewidth=2, label=f"AUC = {roc_auc:.3f}")
    plt.plot([0, 1], [0, 1], linestyle="--", linewidth=1)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(title)
    plt.legend(loc="lower right")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# One-sample visualization (feature heatmap) ----
def plot_sample_feature_heatmap(sample_features, channels=18, bands=8,
                                channel_names=None, band_names=None,
                                title="One Sample: Channels × Bands Feature Heatmap",
                                vmin=None, vmax=None,
                                x_tick_step=None):
    """
    Visualize a single (channels × bands) feature vector as a heatmap.

    sample_features: 1D array-like, length = channels*bands
    channels: number of channels (rows)
    bands: number of bands (columns)
    x_tick_step: show an x-axis label every this many columns (default=None = show all)
    """
    sample_features = np.asarray(sample_features).ravel()
    expected = channels * bands
    if sample_features.size != expected:
        raise ValueError(f"Expected {expected} features, got {sample_features.size}.")

    mat = sample_features.reshape(channels, bands)

    plt.figure(figsize=(8, 6))
    im = plt.imshow(mat, aspect="auto", interpolation="nearest", vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(im)
    cbar.set_label("Feature value", rotation=270, labelpad=12)

    if band_names is None:
        band_names = [f"Band {i+1}" for i in range(bands)]
    if channel_names is None:
        channel_names = [f"Ch {i+1}" for i in range(channels)]

    # control tick step
    if x_tick_step is None:
        x_ticks = np.arange(bands)
    else:
        x_ticks = np.arange(0, bands, x_tick_step)

    plt.xticks(ticks=x_ticks, labels=[band_names[i] for i in x_ticks], rotation=45, ha="right")
    plt.yticks(ticks=np.arange(channels), labels=channel_names)
    plt.xlabel("Frequency Bands")
    plt.ylabel("Channels")
    plt.title(title)
    plt.tight_layout()
    plt.show()


# SVM feature weights heatmap ----
def plot_svm_feature_weights(svm_model, channels=18, bands=8,
                             channel_names=None, band_names=None,
                             use_abs=True,
                             title="SVM Feature Weights (Channels × Bands)"):
    """
    Visualize the learned weights for each of the 144 features (18×8) from a linear SVM.
    - For multiclass (One-vs-Rest), averages |weights| across classes by default (use_abs=True).
    - For binary, uses coef_[0].
    """
    if not hasattr(svm_model, "coef_"):
        raise ValueError("This visualization requires a linear SVM with a 'coef_' attribute (e.g., LinearSVC or SVC(kernel='linear')).")

    coefs = svm_model.coef_
    # coefs shape: (n_classes, n_features)
    if coefs.ndim == 1:
        w = coefs
    else:
        if coefs.shape[0] == 1:
            w = coefs[0]
        else:
            # multiclass: aggregate across classes
            w = np.mean(np.abs(coefs), axis=0) if use_abs else np.mean(coefs, axis=0)

    w = np.asarray(w).ravel()
    expected = channels * bands
    if w.size != expected:
        raise ValueError(f"Expected {expected} weights (channels×bands), got {w.size}.")

    W = w.reshape(channels, bands)

    plt.figure(figsize=(8, 6))
    im = plt.imshow(W, aspect="auto", interpolation="nearest")
    cbar = plt.colorbar(im)
    cbar.set_label("Weight" + (" (|coef| mean)" if coefs.ndim == 2 and coefs.shape[0] > 1 and use_abs else ""), rotation=270, labelpad=12)

    if band_names is None:
        band_names = [f"Band {i+1}" for i in range(bands)]
    if channel_names is None:
        channel_names = [f"Ch {i+1}" for i in range(channels)]

    plt.xticks(ticks=np.arange(bands), labels=band_names, rotation=45, ha="right")
    plt.yticks(ticks=np.arange(channels), labels=channel_names)
    plt.xlabel("Frequency Bands")
    plt.ylabel("Channels")
    plt.title(title)
    plt.tight_layout()
    plt.show()


def top_k_svm_features(svm_model, k=10, feature_names=None):
    """
    Returns indices and weights for the top-k features by absolute weight (binary or averaged multiclass).
    """
    if not hasattr(svm_model, "coef_"):
        raise ValueError("Model has no coef_. Use a linear SVM.")
    coefs = svm_model.coef_
    if coefs.ndim == 2 and coefs.shape[0] > 1:
        w = np.mean(np.abs(coefs), axis=0)
    else:
        w = coefs.ravel()
    order = np.argsort(np.abs(w))[::-1][:k]
    names = [feature_names[i] if feature_names is not None else f"f{i}" for i in order]
    return list(zip(names, w[order]))


In [ ]:
svm_model = trained_models['SVM']
y_pred = svm_model.predict(X_test)

# 1) Confusion matrix
labels = getattr(svm_model, "classes_", None)
plot_confusion_matrix_svm(y_test, y_pred, labels=labels, normalize='true')

# 2) ROC + AUC (binary)
y_score = _get_scores_for_binary_roc(svm_model, X_test)
# Choose the positive label explicitly if your labels aren’t {0,1}
pos_label = 1 if labels is None else labels[1] if len(labels) == 2 else 1

plot_roc_auc_svm(y_test, y_score, pos_label=pos_label)

# 3) One-sample feature heatmap (pick any sample)
plot_sample_feature_heatmap(X_test[0],
                            channels=17, bands=512,
                            x_tick_step=50,  # label every 10 columns
                            channel_names=[f"Ch {i+1}" for i in range(17)])

In [ ]:
# 4) SVM weights (144 features → 18×8)
# plot_svm_feature_weights(svm_model, channels=18, bands=8,
#                          channel_names=[f"Ch {i+1}" for i in range(17)],
#                          band_names=[f"Band {i+1}" for i in range(512)])